## Image generator

In [71]:
import os
import modin.pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from src.model.data_generator import NimaDataGenerator
from src.utils.ava_preprocess import get_ava_csv_df, get_rating_columns

In [2]:
ava_csv_df = get_ava_csv_df()
ava_csv_df.head()

,index,image_id,count_rating_1,count_rating_2,count_rating_3,count_rating_4,count_rating_5,count_rating_6,count_rating_7,count_rating_8,count_rating_9,count_rating_10,tag_1,tag_2,challange_id
0,1,953619,0,1,5,17,38,36,15,6,5,1,1,22,1396
1,2,953958,10,7,15,26,26,21,10,8,1,2,1,21,1396
2,3,954184,0,0,4,8,41,56,10,3,4,0,0,0,1396
3,4,954113,0,1,4,6,48,37,23,5,2,2,15,21,1396
4,5,953980,0,3,6,15,57,39,6,1,1,1,22,38,1396


In [101]:
DATA_DIR = 'data/AVA/images'
image_files = os.listdir(DATA_DIR)
count_columns = get_rating_columns()

df = ava_csv_df[['image_id']]
df['ratings'] = ava_csv_df[count_columns].apply(lambda x : x.values.tolist(), axis=1)
# count_columns
df.head()

,image_id,ratings
0,953619,"[0, 1, 5, 17, 38, 36, 15, 6, 5, 1]"
1,953958,"[10, 7, 15, 26, 26, 21, 10, 8, 1, 2]"
2,954184,"[0, 0, 4, 8, 41, 56, 10, 3, 4, 0]"
3,954113,"[0, 1, 4, 6, 48, 37, 23, 5, 2, 2]"
4,953980,"[0, 3, 6, 15, 57, 39, 6, 1, 1, 1]"


In [135]:
a = np.empty((2, 2,2))
b= np.random.random((2,2))
print(a)
a[0] = b
print(a)

[[[0.24287354 0.92724093]
  [0.53438362 0.97344584]]

 [[0.         0.        ]
  [0.         0.        ]]]
[[[0.10996128 0.45419193]
  [0.46047498 0.24746346]]

 [[0.         0.        ]
  [0.         0.        ]]]


In [119]:
batch_size = 32
index = 2
indexes = np.arange(len(df))
batch_indexes = indexes[index*batch_size: (index+1)*batch_size]
batch_sample = df.iloc[batch_indexes]
# batch_indexes
batch_sample[:5]

,image_id,ratings
64,954180,"[0, 0, 3, 14, 35, 51, 10, 3, 1, 1]"
65,954014,"[0, 3, 7, 26, 57, 19, 2, 1, 1, 3]"
66,954092,"[0, 0, 3, 1, 26, 42, 30, 18, 6, 4]"
67,954226,"[9, 11, 24, 28, 28, 11, 5, 5, 0, 0]"
68,953946,"[0, 0, 4, 4, 34, 56, 14, 8, 5, 0]"


In [97]:
npl = np.array(df.iloc[0,1])
npl/npl.sum()

array([0.        , 0.00806452, 0.04032258, 0.13709677, 0.30645161,
       0.29032258, 0.12096774, 0.0483871 , 0.04032258, 0.00806452])

In [106]:
data_generator = NimaDataGenerator(df, img_directory = DATA_DIR, x_col='image_id', y_col='ratings', preprocess_input=None, batch_size=32)

Found 15084 valid image filenames belonging to 10 classes.


In [107]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(df, test_size=0.1)
df_train.head()

,image_id,ratings
5157,101759,"[8, 22, 36, 80, 80, 55, 14, 8, 2, 1]"
4089,112059,"[7, 7, 35, 48, 70, 56, 21, 9, 8, 3]"
13521,103148,"[0, 1, 10, 38, 87, 46, 20, 1, 2, 0]"
326,135448,"[4, 10, 29, 43, 45, 31, 10, 5, 2, 1]"
4539,13078,"[18, 36, 36, 50, 36, 15, 10, 1, 2, 2]"


In [109]:
# Import the libraries 
from tensorflow.keras.layers import Dropout, Flatten, MaxPool2D, Conv2D, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from src.model.loss import earth_movers_distance

# model = Sequential()
# model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(224, 224, 3)))
# model.add(MaxPool2D(pool_size=2, strides=2))
# model.add(Dropout(0.2))

# model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(224, 224, 3)))
# model.add(MaxPool2D(pool_size=2, strides=2))
# model.add(Dropout(0.2))

# model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(224, 224, 3)))
# model.add(MaxPool2D(pool_size=2, strides=2))
# model.add(Dropout(0.2))

# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dense(10, activation='softmax'))
# model.summary()

model.compile(optimizer=Adam(), loss=earth_movers_distance)

In [114]:
train_datagen = NimaDataGenerator(
    df_train,
    img_directory=DATA_DIR,
    x_col="image_id",
    y_col="ratings",
    preprocess_input=None,
    batch_size=32,
    input_size=(256, 256, 3),
    crop_size=(224, 224),
)

valid_datagen = NimaDataGenerator(
    df_valid,
    img_directory=DATA_DIR,
    x_col="image_id",
    y_col="ratings",
    preprocess_input=None,
    batch_size=32,
    input_size=(256, 256, 3),
    crop_size=(224, 224),
)

earlystopping = EarlyStopping(
    min_delta=0.001, monitor="val_loss", patience=4, restore_best_weights=True
)

CKPT_DIR = "src/weights"
checkpoint = ModelCheckpoint(
    filepath=CKPT_DIR,
    save_weights_only=True,
    monitor="val_loss",
    mode="max",
    save_best_only=True,
)

Found 13575 valid image filenames belonging to 10 classes.
Found 1509 valid image filenames belonging to 10 classes.


In [118]:
# Train and fit model
history = model.fit(
    train_datagen,
    validation_data=valid_datagen,
    callbacks=[earlystopping, checkpoint],
#     use_multiprocessing=True,
    epochs=50,
    verbose=1,
)


TypeError: 'numpy.float64' object cannot be interpreted as an integer